In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, LSTM, Masking
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

/opt/anaconda3/envs/tf_env/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
# Set the path to the dataset
datasetspath = "/Users/saransathy/WalshDBA/Capstone"

In [3]:
df = pd.read_pickle(f"{datasetspath}/battery_dataset1_normalised.pkl")

In [4]:
cols1 = ['mileage', 'capacity','Curr','SoC','Temp','Volt']
cols2 = ['Curr','SoC','Temp','Volt']

In [5]:
# Prepare Data
sequences = []
labels = []
for name, group in df.groupby(['car','charge_segment']):
    seq = group[cols2].values
    sequences.append(seq)
    # Use target of first row in each group for label
    label = 0
    if (group['label'].iloc[0] == '10'):
        label = 1
    labels.append(label)

In [7]:
# Pad sequences to the same length (maxlen)
X = pad_sequences(sequences, dtype='float32', padding='post', value=0.0)
y = np.array(labels)

In [8]:
# Release Memory
del df, sequences, labels

In [9]:
from tensorflow.keras.models import load_model

In [10]:
model = load_model(f"{datasetspath}/lstm_model.keras")

/opt/anaconda3/envs/tf_env/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 7 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [11]:
# Print DL Model Results
def dl_model_results(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5).astype(int)
    model_accuracy = accuracy_score(y_test, y_pred)
    model_classification_report = classification_report(y_test, y_pred)
    model_confusion_matrix = confusion_matrix(y_test, y_pred)
    print(f"Accuracy: {model_accuracy:.4f}")
    print("Classification Report:")
    print(model_classification_report)
    print("Confusion Matrix:")
    print(model_confusion_matrix)
    del y_pred
    return model_accuracy, model_classification_report, model_confusion_matrix

In [12]:
dl_model_results(model, X, y)

3035/3035 ━━━━━━━━━━━━━━━━━━━━ 390s 128ms/step
Accuracy: 0.1784
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.12      0.22     89824
           1       0.08      0.88      0.14      7275

    accuracy                           0.18     97099
   macro avg       0.50      0.50      0.18     97099
weighted avg       0.86      0.18      0.21     97099

Confusion Matrix:
[[10925 78899]
 [  875  6400]]


(0.17842614239075583,
 '              precision    recall  f1-score   support\n\n           0       0.93      0.12      0.22     89824\n           1       0.08      0.88      0.14      7275\n\n    accuracy                           0.18     97099\n   macro avg       0.50      0.50      0.18     97099\nweighted avg       0.86      0.18      0.21     97099\n',
 array([[10925, 78899],
        [  875,  6400]]))